In [2]:
import pandas as pd


## Load Clusters and Alarms

In [3]:
real_time_clusters_df = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters.parquet")

In [8]:
filtered_tx_alarms_adsl_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')

In [7]:
filtered_tx_alarms_pdh_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')

In [6]:
filtered_tx_alarms_ptn_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')


In [5]:
filtered_mob_alarms_df = pd.read_parquet('alarms datasets/mob/[filtered]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

## Extract Last Occurence field form Alarms

In [9]:
adsl_last_occurence = filtered_tx_alarms_adsl_df.loc[:, ['alarm_id', 'last_occurrence']]
pdh_last_occurence = filtered_tx_alarms_pdh_df.loc[:, ['alarm_id', 'last_occurrence']]
ptn_last_occurence = filtered_tx_alarms_ptn_df.loc[:, ['alarm_id', 'last_occurrence']]
mob_last_occurence = filtered_mob_alarms_df.loc[:, ['alarm_id', 'last_occurrence']]


In [10]:
all_alarms = pd.concat([adsl_last_occurence, pdh_last_occurence, ptn_last_occurence, mob_last_occurence])

In [12]:
all_alarms.to_parquet('alarms datasets/alarms_last_occurence.parquet')

## Join Clusters with Last Occurence

In [4]:
all_alarms = pd.read_parquet('alarms datasets/alarms_last_occurence.parquet')

In [5]:
clusters_merge_df = pd.merge(real_time_clusters_df, all_alarms, on='alarm_id', how='left')

In [6]:
clusters_merge_df

,alarm_id,network,ne_type,first_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2,last_occurrence
0,EFM;63af9dfe2aaf2c000125807a,an,lnbts,2022-12-31 03:27:10,2022-12-31 23:59:01,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1,2023-01-02 13:52:56
1,EFM;63af9dfec623b40001a5c9cf,an,lnbts,2022-12-31 03:27:10,2022-12-31 23:58:55,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1,2023-01-02 13:52:56
2,COS1P;136902468,sdh,ad16,2022-12-31 22:19:34,2022-12-31 23:58:46,2023-01-01 00:00:02,alesitaj/ad16/73,ales,2022-12-31 22:19:34,tx,None,202301010000_20,202301010000_2,NaT
3,EFM;63b0beaf2aaf2c0001258736,an,lebts,2022-12-31 23:58:55,2022-12-31 23:59:27,2023-01-01 00:00:02,5btsle119,alss,2022-12-31 23:58:55,mob,None,202301010000_9,202301010000_3,2023-01-01 00:08:06
4,COS1P;136891403,sdh,oadm,2022-12-31 11:26:16,2022-12-31 23:59:52,2023-01-01 00:00:02,ancoitab/oadm/1,anco,2022-12-31 11:26:16,tx,None,202301010000_21,202301010000_4,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123609,EFMTX;6591efb9d7d6bc00019444d8,adsl,adsl,2023-12-31 23:48:25,2023-12-31 23:56:43,2023-12-31 23:58:02,taglitac/adsl/1,tagl,2023-12-31 23:48:24,tx,None,202312312350_16,202312312350_13,2023-12-31 23:52:28
13123610,EFMTX;6591f046d8beb800016deaef,ptn,ptn3,2023-12-31 23:50:46,2023-12-31 23:57:34,2023-12-31 23:58:02,thesitae/ptn3/1,thes,2023-12-31 23:44:44,tx,None,202312312348_17,202312312344_9,2023-12-31 23:50:46
13123611,EFM;6591f199d4c961000177739d,an,lnbts,2023-12-31 23:56:25,2023-12-31 23:56:37,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13,2023-12-31 23:58:21
13123612,EFM;6591f19aba09b0000160c08d,an,lnbts,2023-12-31 23:56:26,2023-12-31 23:57:59,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13,2023-12-31 23:58:21


In [13]:
cluster_merge_without_sdh = clusters_merge_df[clusters_merge_df['network'] != "sdh"]
cluster_merge_without_sdh.insert(4, 'last_occurrence', cluster_merge_without_sdh.pop('last_occurrence'))
cluster_merge_without_sdh.to_parquet('real-time clusters/20230101-20240101_real_time_clusters_with_last_occurence.parquet')

## Extract Cluster from Datasets

In [ ]:
gruppi = real_time_clusters_df.groupby(['cluster_id', 'cluster_id2'])
clusters = []

# Itera sui gruppi e salva i cluster in una nuova struttura dati
for nome_gruppo, gruppo in gruppi:
    clusters.append(gruppo)

In [ ]:
gruppi_count = gruppi.size().reset_index(name='size')

In [ ]:
gruppi_count.sort_values(by="size", ascending=False)

## Clusters Statistics

- Size: number of alarms within it
- Duration/lifespan: difference in time from the last last occurrence to the first first occurrence
- Delta first occurrence: distance between the last available first occurrence and the first ever first occurrence